In [251]:
import sqlalchemy as sq

engine = sq.create_engine("postgresql://localhost/crime2021")
conn = engine.connect()

In [252]:
statement = """select nibrs_offense.offense_id, a.county_name, ni.data_year, ni.incident_hour, ni.incident_date, nbm.bias_id, nlt.location_name, n.offense_name, n.offense_code, n.offense_category_name, nwt.weapon_code from nibrs_offense
                JOIN nibrs_bias_motivation nbm on nibrs_offense.offense_id = nbm.offense_id
                left join nibrs_weapon nw on nibrs_offense.offense_id = nw.offense_id
                left join nibrs_weapon_type nwt on nw.weapon_id = nwt.weapon_id
                join nibrs_incident ni on nibrs_offense.incident_id = ni.incident_id
                join nibrs_bias_list nbl on nbm.bias_id = nbl.bias_id
                join agencies a on ni.agency_id = a.agency_id
                join nibrs_location_type nlt on nibrs_offense.location_id = nlt.location_id
                join nibrs_offense_type n on nibrs_offense.offense_code = n.offense_code;"""
result = conn.execute(statement)
conn.close()

In [253]:
SEX_CRIME_CODE = ["11", "36", "90H"]            # rape, sodomy, SA with object, fondling, incest, stat. rape, peeping tom - offense_code
FIREARM_CODE   = ["11", "12", "13", "14", "15"] # NIBRS_WEAPON_TYPE, firearm, handgun, rifle, shotgun, other firearm - weapon_code
HATE_CODE      = "88"                           # if result contains this number, it is NOT a hate crime - bias_id

def create_incident_time_dict(incident: dict) -> dict:
    time_dict = dict()
    time_dict["inc_hour"] = incident["incident_hour"]
    time_dict["inc_day"] = incident["incident_date"].day
    time_dict["inc_month"] = incident["incident_date"].month
    time_dict["inc_year"] = incident["incident_date"].year
    return time_dict

def awful_array_string_checker(string: str, arr: list) -> bool: # I feel like there's a better built in function for this... oh well
    for e in arr:
        if e in string:
            return True
    return False

def check_codes(incident: dict, finished_dict: dict) -> dict:
    finished_dict['gun_violence'] = False
    finished_dict['sex_crime']    = False
    finished_dict['hate_crime']   = False

    if awful_array_string_checker(incident['offense_code'], SEX_CRIME_CODE):
        finished_dict['sex_crime'] = True
    if (wc := incident['weapon_code']) and awful_array_string_checker(wc, FIREARM_CODE):
        finished_dict['gun_violence'] = True
    if HATE_CODE not in str(incident['bias_id']):
        finished_dict['hate_crime'] = True

    return finished_dict

In [254]:
def process_results(res):
    import time
    start = time.time()
    res_dict = dict()
    seen = dict()
    i = 0
    for u in res.all():
        #if i == 1: break;
        incident_dict = dict(u)

        finished_incident_dict = dict()
        # add timing
        time_dict = create_incident_time_dict(incident_dict)
        finished_incident_dict["inc_time"] = time_dict

        # check crime, offense, bias codes to generate boolean
        finished_incident_dict = check_codes(incident_dict, finished_incident_dict)

        finished_incident_dict["crime_desc"] = incident_dict['offense_name']
        finished_incident_dict['off_code'] = incident_dict['offense_code']
        finished_incident_dict['loc_id'] = incident_dict['location_name']
        # strip this to make index
        finished_incident_dict['county'] = incident_dict['county_name']
        finished_incident_dict['id'] = incident_dict['offense_id']

        # if another incident exists with another offender, keep code flags
        if incident_dict['offense_id'] in seen:
            continue
        else:
            seen[incident_dict['offense_id']] = incident_dict

        res_dict[i] = finished_incident_dict
        i+=1
    # import pprint
    # pprint.pprint(res_dict)
    end = time.time()
    print(f"Done processing {len(res_dict)} rows in {end-start} seconds")
    return res_dict


In [255]:
def multiple_county_handler(entry, f_handler, idx, county_sep: str):
    county = entry['county']
    # print(f"Handling multiple counties {county}")

    curr_idx = county.index(county_sep, idx)+2
    next_idx = -1

    try:
        next_idx = county.index(county_sep, curr_idx)
    except ValueError:
        pass
    if next_idx != -1:
        idx = next_idx

    index_dict = dict()
    sub_index_dict = dict()
    curr_county = county[curr_idx: next_idx if next_idx != -1 else len(county)]
    sub_index_dict['_index'] = curr_county.replace(" ", ",").lower()
    # sub_index_dict['dupe'] = True
    old_county = entry.pop('county')
    # del entry['county']

    sub_index_dict['_id'] = entry['id']
    # del entry['id']
    old_id = entry.pop('id')

    index_dict['index'] = sub_index_dict

    metadata = json.dumps(index_dict)
    data = json.dumps(entry)

    entry['county'] = old_county
    entry['id'] = old_id

    f_handler.write(f'{metadata}\n')
    f_handler.write(f'{data}\n')

    return idx

def write_data(res_dict: dict, county_sep: str):
    import time
    start = time.time()
    lines = 0
    chunks = 0
    import json
    import uuid
    f = open(f'data/{uuid.uuid4()}.json', 'w', encoding='utf-8')
    for entry in res_dict.values():
        index_dict = dict()
        sub_index_dict = dict()
        county = entry['county'] # type: str
        index_found = 0
        for i in range(county.count(county_sep)): # multiple counties check
            index_found = multiple_county_handler(entry, f, index_found, county_sep)

        stopping_point = len(county) if county_sep not in county else county.index(county_sep)
        county = county[0:stopping_point]
        county = county.replace(" ", "_").lower()
        sub_index_dict['_index'] = county
        del entry['county']

        sub_index_dict['_id'] = entry['id']
        del entry['id']

        index_dict['index'] = sub_index_dict

        metadata = json.dumps(index_dict)
        data = json.dumps(entry)

        f.write(f'{metadata}\n')
        f.write(f'{data}\n')
        lines +=1
        if lines % 200 == 0:
            f.close()
            f = open(f'data/{uuid.uuid4()}.json', 'w', encoding='utf-8')
            chunks += 1
    f.close()
    stop = time.time()
    print(f'Finished writing {lines} entries to {chunks} chunks in {stop-start} seconds')

In [256]:
import sqlalchemy as sq

engine = sq.create_engine("postgresql://localhost/crime2021")
conn = engine.connect()
statement_2021 = """select nibrs_offense.offense_id, a.county_name, ni.data_year, ni.incident_hour, ni.incident_date, nbm.bias_id, nlt.location_name, n.offense_name, n.offense_code, n.offense_category_name, nwt.weapon_code from nibrs_offense
                JOIN nibrs_bias_motivation nbm on nibrs_offense.offense_id = nbm.offense_id
                left join nibrs_weapon nw on nibrs_offense.offense_id = nw.offense_id
                left join nibrs_weapon_type nwt on nw.weapon_id = nwt.weapon_id
                join nibrs_incident ni on nibrs_offense.incident_id = ni.incident_id
                join nibrs_bias_list nbl on nbm.bias_id = nbl.bias_id
                join agencies a on ni.agency_id = a.agency_id
                join nibrs_location_type nlt on nibrs_offense.location_id = nlt.location_id
                join nibrs_offense_type n on nibrs_offense.offense_code = n.offense_code;"""
result_2021 = conn.execute(statement_2021)
conn.close()

engine = sq.create_engine("postgresql://localhost/crime")
conn = engine.connect()
statement_all_other_years = """select nibrs_offense.offense_id, a.county_name, ni.data_year, ni.incident_hour, ni.incident_date, nbl.bias_code as bias_id, nlt.location_name, n.offense_name, n.offense_code, n.offense_category_name, nwt.weapon_code from nibrs_offense
                                JOIN nibrs_bias_motivation nbm on nibrs_offense.offense_id = nbm.offense_id
                                left join nibrs_weapon nw on nibrs_offense.offense_id = nw.offense_id
                                left join nibrs_weapon_type nwt on nw.weapon_id = nwt.weapon_id
                                join nibrs_incident ni on nibrs_offense.incident_id = ni.incident_id
                                join nibrs_bias_list nbl on nbm.bias_id = nbl.bias_id
                                join agencies a on ni.agency_id = a.agency_id
                                join nibrs_location_type nlt on nibrs_offense.location_id = nlt.location_id
                                join nibrs_offense_type n on nibrs_offense.offense_type_id = n.offense_type_id;"""
result_others = conn.execute(statement_all_other_years)
conn.close()

In [1]:
#test code for faster filtering
# import sqlalchemy as sq

# def execute_statement(engine_url: str, statement: str):
#     engine = sq.create_engine(engine_url)
#     with engine.connect() as conn:
#         result = conn.execute(statement)
#     return result

# statement_2021 = """select nibrs_offense.offense_id, a.county_name, ni.data_year, ni.incident_hour, ni.incident_date, nbm.bias_id, nlt.location_name, n.offense_name, n.offense_code, n.offense_category_name, nwt.weapon_code from nibrs_offense
#                 JOIN nibrs_bias_motivation nbm on nibrs_offense.offense_id = nbm.offense_id
#                 left join nibrs_weapon nw on nibrs_offense.offense_id = nw.offense_id
#                 left join nibrs_weapon_type nwt on nw.weapon_id = nwt.weapon_id
#                 join nibrs_incident ni on nibrs_offense.incident_id = ni.incident_id
#                 join nibrs_bias_list nbl on nbm.bias_id = nbl.bias_id
#                 join agencies a on ni.agency_id = a.agency_id
#                 join nibrs_location_type nlt on nibrs_offense.location_id = nlt.location_id
#                 join nibrs_offense_type n on nibrs_offense.offense_code = n.offense_code;"""
# result_2021 = execute_statement("postgresql://localhost/crime2021", statement_2021)

# statement_all_other_years = """select nibrs_offense.offense_id, a.county_name, ni.data_year, ni.incident_hour, ni.incident_date, nbl.bias_code as bias_id, nlt.location_name, n.offense_name, n.offense_code, n.offense_category_name, nwt.weapon_code from nibrs_offense
#                                 JOIN nibrs_bias_motivation nbm on nibrs_offense.offense_id = nbm.offense_id
#                                 left join nibrs_weapon nw on nibrs_offense.offense_id = nw.offense_id
#                                 left join nibrs_weapon_type nwt on nw.weapon_id = nwt.weapon_id
#                                 join nibrs_incident ni on nibrs_offense.incident_id = ni.incident_id
#                                 join nibrs_bias_list nbl on nbm.bias_id = nbl.bias_id
#                                 join agencies a on ni.agency_id = a.agency_id
#                                 join nibrs_location_type nlt on nibrs_offense.location_id = nlt.location_id
#                                 join nibrs_offense_type n on nibrs_offense.offense_type_id = n.offense_type_id;"""
# result_others = execute_statement("postgresql://localhost/crime", statement_all_other_years)


In [257]:

import shutil
shutil.rmtree('data', ignore_errors=True)
import os
os.mkdir('data')
res_2021_dict = process_results(result_2021)
write_data(res_2021_dict, ", ")
res_others_dict = process_results(result_others)
write_data(res_others_dict, "; ")

Done processing 390517 rows in 2.1734089851379395 seconds
Finished writing 390517 entries to 1952 chunks in 2.406928062438965 seconds
Done processing 1625858 rows in 13.483729839324951 seconds
Finished writing 1625858 entries to 8129 chunks in 9.574924945831299 seconds


In [258]:
# import os
# os._exit(1)